<a href="https://colab.research.google.com/github/aarav-2/Deep-Learning-assignments/blob/main/Day5_Task_LSTM_LM_handout_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM in practice -- NLP

## Language modeling

A language model is a probability distribution over the sequence of words, modeling language (production), thus if the set of words is $w$, then for arbitrary $\mathbf w = \langle w_1,\dots, w_n\rangle$ ($w_i\in W$) sequence it defines a $P(\mathbf w)$ probability.

Probability with chain rule:

$$P(\mathbf w)= P(w_1)\cdot P(w_2 \vert w_1 )\cdot P(w_3\vert w_1, w_2)\cdot\dots\cdot P(w_n\vert w_1,\dots, w_{n-1})$$

$$P(This\;is\;a\;nice\;house)= P(This)\cdot P(is \vert This )\cdot P(a\vert This, is) \cdot P(nice\vert This, is, a)\cdot P(house\vert This, is, a, nice)$$

so this means, that for the modeling we need only to give the conditional probability of the "continuation", the next word, thus for $w$ word and $\langle w_1,\dots,w_n\rangle$ sequence the probability that the next word will be $w$

$$P(w ~\vert ~ w_1,\dots,w_n)$$

There are character based models also, which take the individual characters as units, not the words, and model language as a distribution over sequences of characters (think T9...)

### Measurement of performance: Perplexity

First, we should split our original corpus $w=\langle w_1,\dots, w_n\rangle$ to train $w_{train}=\langle w_1,\dots, w_k\rangle$ and test data $w_{test}=\langle w_{k+1},\dots, w_n\rangle$

A language model $\mathcal M$'s perplexity over the word series $\mathbf w_{test} = \langle w_{k+1},\dots, w_n\rangle$ is:

$$\mathbf{PP}_{\mathcal M}(\mathbf w_{test}) = \sqrt[n-k]{\frac{1}{P_{\mathcal M}(\mathbf w_{test})}}$$

With the chain rule can be rewritten as:

$$\mathbf{PP}_{\mathcal M}(\mathbf w_{test}) = {\sqrt[n-k]{\frac{1}{P_{\mathcal M}(w_{k+1})}\cdot \frac{1}{P_{\mathcal M}(w_{k+2} \vert w_{k+1} )}\cdot \frac{1}{P_{\mathcal M}(w_{k+3}\vert w_{k+1}, w_{k+2})}\cdot\dots\cdot \frac{1}{P_{\mathcal M}(w_n\vert w_{k+1},\dots, w_{n-1})}}}$$

which is exactly the geometric mean of the reciprocals of the conditional probabilities of all words in the corpus.

In case of a bigram model this is further simplified to:
$$\mathbf{PP}_{\mathcal M}(\mathbf w_{test}) = \sqrt[n-k]{\frac{1}{P_{\mathcal M}(w_{k+1})}\cdot \frac{1}{P_{\mathcal M}(w_{k+2} \vert w_{k+1} )}\cdot \frac{1}{P_{\mathcal M}(w_{k+3}\vert w_{k+2})}\cdot\dots\cdot \frac{1}{P_{\mathcal M}(w_n\vert w_{n-1})}}$$

Example for a bigram model perplexity:
- **Train corpus**: "This is the house that Jack built. What a house. Jack is a genius."
- **Test corpus**: "This is a house."

$$\mathbf{PP}_{\mathcal M}(This\;is\;a\;house) = \sqrt[4]{\frac{1}{P_{\mathcal M}(This)}\cdot \frac{1}{P_{\mathcal M}(is \vert This)}\cdot \frac{1}{P_{\mathcal M}(a \vert is)}\cdot \frac{1}{P_{\mathcal M}(house\vert a)}}=$$

$$ = \sqrt[4]{\frac{1}{0.33}\cdot \frac{1}{1}\cdot \frac{1}{0.5}\cdot \frac{1}{0.5}} \approx 1.86$$

The **lower** perplexity the better.

[source](https://www.youtube.com/watch?v=oaYsCVtHveQ)

### But what is it good for?
For example:
- Predictive text input ("autocomplete")
- Generating text
- Spell checking
- Language understanding
- And most importantly representation learning - this we will be studiying in detail in a next lecture

### Generating text with a language model

The language model produces a tree with probable continuations of the text:

<img src="https://4.bp.blogspot.com/-Jjpb7iyB37A/WBZI4ImGQII/AAAAAAAAA9s/ululnUWt2vw9NMKuEr-F9H8tR2LEv36lACLcB/s1600/prefix_probability_tree.png" width=400 heigth=400>

Using this tree we can try different algorithms to search for the best "continuations". A full breadth-first search oi usually impossible, due to the high branching factor of the tree.

Alternatives:
- "Greedy": we choose the continuation which has the highest direct probability, This will most probably be suboptimal, since the probability of the full sequence is tha product of the continuations, and if we would have chosen a different path, we might ahve been able to choose later words with hihg probabilities.
- Beam-search: we always store a fixed $k$ number of partial sequences, and we always try to expand these, always keeping the most probable $k$ from the possible continuations.

Example ($k$=5):

<img src="http://opennmt.net/OpenNMT/img/beam_search.png" width=600 heigth=600>


### The "old way": N-gram based solutions

With _gross_ simplification we assume, that the distribution is only dependent on the prior $n-1$ words (where $n$ is typically $<=4$), thus we assume a Markov chain of the order $n$:

 $$P(w ~\vert ~ w_1,\dots,w_k) = P(w ~\vert ~ w_{k- n + 2},\dots,w_k)$$

Example for 2-gram or Bigram language model:

$$Toy\;corpus:This\;is\;the\;malt.\;That\;lay\;in\;the\;house\;that\;Jack\;built.$$

$$P(This\;is\;the\;house) = P(This)\cdot P(is\vert This)\cdot P(the\vert is)\cdot P(house\vert the)$$

[Source](https://www.youtube.com/watch?v=GiyMGBuu45w)

We simply compute these probabilities in a frequentist style by calculating the $n$-gram statistics of the corpus at hand:

$$P(w_2 ~\vert ~w_1) = \frac{c(\langle w_1, w_2 \rangle)}{c(w_1)}$$

$$P(Jack \vert that)=\frac{c(that, Jack)}{c(that)}=\frac{1}{2}=50\%$$

$$P(w_{k+1} \vert~ w_1,\dots,w_k)_\mathrm = \frac{c(\langle w_1,...,w_k, w_{k+1} \rangle)}{c(\langle w_1, \dots w_k\rangle)}$$

Please note, that in this case we are using "memorization", a form of database learning, with minimal compression - "counting".

But what do we do the given $n$-grams rarely or never occur? We have to employ some __smoothing__ solutions, like:

##### Additive smoothing
We pretend that we have seen the $n$-grams more times than we have actually did with a fixed $\delta$ number, in the simplest case with $n=2$:

$$P(w_2 ~\vert ~w_1) = \frac{c(\langle w_1, w_2 \rangle) + \delta}{\sum_{w\in V} [c(\langle w_1, w\rangle) + \delta]}$$

Widespread solution for $\delta$ is $1$.

The main problem with this kind of smoothing is that it does not take into account by "supplementing" the data the frequency of components of shorter $n$-grams, eg. if neither $\langle w_1, w_2 \rangle$  nor $\langle w_1, w_3 \rangle$ occurs in the corpus, it assumes the frequency of both bigrams to be $\delta$, irrespective of the ratio of frequencies of $w_2$ and $w_3$.

Most smoothing techniques are trying to accomodate this, eg: simple interpolation:

##### Interpolatcion

In case of bigrams, we add - with a certain weight - the probabilities coming from the individual frequencies:

$$P(w_2 ~\vert ~w_1)_{\mathrm{interp}} = \lambda_1\frac{c(\langle w_1, w_2 \rangle)}{c(w_1)} + (1 - \lambda_1)\frac{c(w_1)}{\sum_{w\in V}c(w)}$$

Recursive solution for arbitrary $k$:

$$P(w_{k+1} \vert~ w_1,\dots,w_k)_\mathrm{interp} = \lambda_k\frac{c(\langle w_1,...,w_k, w_{k+1} \rangle)}{c(\langle w_1, \dots w_k\rangle)} + (1-\lambda_k)P_\mathrm{interp}(\langle w_2,\dots,w_{k+1}\rangle)$$

$\lambda_k$ is empirically set by examining the corpus, typically by [Expectation Maximization algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), which - as we have mentioned - iteratively tunes the parameters to maximize the likelihood.


Good overview about the smoothing methods: [MacCartney, NLP Lunch Tutorial: Smoothing](https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf)


#### General problems

- Even the core assumption is not too realistic, since the probabilities are for sure influenced in a way by words further than $n$, but for practical reasons, it has to be limited (sparsity, computation capacity).
- On a large enough corpus, the memory footprint of the $n$-gram models is _huge_, eg. for the 1T n-gram corpus of Google ([see here](https://catalog.ldc.upenn.edu/LDC2006T13)) containing 1,024,908,267,229 tokens the $n$-gram counts are as follows:
    - unigram: 13,588,391,
    - bigram: 314,843,401,
    - trigram: 977,069,902,
    - fourgrams: 1,313,818,354
    - fivegram: 1,176,470,663.

## Language modeling with LSTMs

One way to circumvent the Markov assumption is to use RNN-s, which are capable of modeling the long-ter dependencies inside the sequence of words. The text is thus considered to be a time-series, and thus an appropriate architecture can be used (as we have already seen):

<img src="http://drive.google.com/uc?export=view&id=1y8QYr9ftTvXAxgzS-ldnGlijVpmK2l21" width=600 heigth=600>



Notable features:

- Input is a "one-hot" encoded vector, wchic we on the spot transform into an "embedding vector"
- For each output step, we get a probability distribution over the whole vocabulary with softmax
- This above is a simple RNN, but LSTMs can be used without any problems

### Teaching

_In theory_ an RNN could be trained with full GD on the corpus in one go:

<img src="http://drive.google.com/uc?export=view&id=1XsBoRp7cNay3svFLRDv2JEDyC7m7CUdC" width=600 heigth=600>


- The loss is generally the well-kown crossentropy, which is in this case (since the input is a one-hot vector):
  $$J^{(i)}(\Theta) = -\log (\hat y[x^{(i+1)}])$$
  the negative logarithm of the probability assigned by the network to the right word / next word.

- For the sake of more frequent updates, and since BPTT for long sequences is very expensive, teaching is done in smaller units with not necessarily the same length.
- The unit is typically one or more sentece, or if the length allows, and we have enough material, a paragraph can be a good candidate.
- Initial state in case of the time-series units: if the boundaries are inside a unit of text, it is important to _transfer the hidden state_ from the previous unit, in other cases initialization can be done by some fixed value.
- (Somewhat misleading) terminology: the length of the "time" unit is _time step_, but sometimes certain implementations call it _minibatch_, though that would generally mean the number of units processed in one go for the sake of computaitonal efficiency.


### LSTM as layers

+ An LSTM - how ever strange that may sound - can be considered to be a complete layer. The most important parameter of it is the "number of (memory) units", which is the length of the hidden state vector, thus, the memory capacity. **Warning: this does not have any relationship to input size, thus can be considered a freely chosen parameter.**
+ It is quite widespread to use multiple LSTM layers ("stacked LSTMs") -- as in the case of ConvNets the hope is, that the layers learn a hierarchy of abstract representations:

<img src="http://wenchenli.github.io/assets/img/GNMT_residual.png" width=60%>

(on the right side a network is shown with skip/residual connections!)

In this case it makes sense, that we do not only get on top of the LSTM a final prediction $h$ (or even prediction + inner state vector $c$) for a sequence, but **we ask it to output the whole sequence of predictions**, so that the next layer can also operate on full sequences. Please bear this in mind during implementation, since this can be a common source of failure.  



## An LSTM language model in Keras

For this task the inspiration comes from the famous [reference work of Andrej Karpathy](https://karpathy.github.io/2015/05/21/rnn-effectiveness/).

Note, that in this case we will not use regularization, since we are willing to overfit - for the sake of play with the text. This is now an "overfitting competition", so _not_ a generally good practice!

## Reader

In [1]:
import numpy as np
import tensorflow as tf
import nltk

from numpy.random import seed
seed(1212)

tf.random.set_seed(1234)

nltk.download("brown")

from nltk.corpus import brown

# This can be an important parameter, so be aware of it...
max_seq_length = 15
max_num_of_sents = 57200
# max_num_of_sents = 50 # How many sentences should we read from the corpus (max=57200)

def generate_brown_word_to_id_map():
    """Return a dictionary mapping downcased Brown-words to their ids.
    Numbering starts from 1 since we use 0 for masking (!!!).
    """
    words = set()
    for word in brown.words():
        words.add(word.lower())
    return {word: idx + 1 for idx, word in enumerate(sorted(words))}


class BrownReader:
    """A reader class for the Brown corpus.
    """

    def __init__(self):
        self.word_to_id_map = generate_brown_word_to_id_map()
        self.id_to_word_map = {idx: word for word, idx in self.word_to_id_map.items()}

    def n_words(self):
        return len(self.word_to_id_map)

    def sentence_to_ids(self, sentence):
        """Return the word ids of a sentence.
        """
        return [self.word_to_id_map[word.lower()] for word in sentence]

    def sentences(self):
        """Generator yielding features from the Brown corpus.
        """
        return (self.sentence_to_ids(sentence) for sentence in brown.sents())

    def sentence_matrixes(self):
        x = np.zeros((max_num_of_sents, max_seq_length-1))
        y = np.zeros((max_num_of_sents, max_seq_length-1))
        sents = self.sentences()
        for idx, sent in enumerate(sents):
            if idx == max_num_of_sents:
                break
            np_array = np.asarray(sent)
            length  = min(max_seq_length, len(np_array))
            x[idx, :length - 1] = np_array[:length - 1]
            y[idx, :length - 1] = np_array[1:length]
        return x, y


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


## Model

### Parameters

In [2]:
br = BrownReader()
n_words = br.n_words()

max_input_length = max_seq_length - 1 # since our x/y input does not contain the last/first element of the sentences

In [3]:
data_x, data_y = br.sentence_matrixes()

In [4]:
data_y = np.expand_dims(data_y, -1) # It seems that Keras needs this for the "one-cold" and softmax dims to match

# Tasks

See below

In [5]:
# Network parameters

lstm_size = 128
embedding_size = 100


### Network

In [6]:
# Import:
# Import the appropriate layers from tf.keras!
# Think about it, that one layer should map ("embed") tha input into a dense vector!
# Don't forget the main model class - according to functional or sequential API
# And eventually the optimizer and backend
# Later one for resetting the graph - good practice!
# And think about it what loss function you will use. The output is a classification
# (categorical) task, it is sparse, so...
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K


# Please reset the graph here!
K.clear_session()

# Model
########
# Build the model!
# Start with the input layer, it receives a vector of the length of the maximal text span (sentence)
# And: vector or not, shape is a tuple...
input_layer = Input(shape=(max_input_length,))

# After this, use the "mapping" layer.
# WARNING:
# 1. width = number of words +1
# 2. it's size is defined in a parameter, somewhere above
# 3. length of input: max input size -1
# 4. Zero values are to be masked in it!!!
embedding_layer = Embedding(input_dim=n_words + 1, output_dim=embedding_size, input_length=max_input_length, mask_zero=True)(input_layer)
#    the constructor has a named argument which has to be given with the True value!!!
# TASK: Can you please verbally elaborate to the instructor, why the points above are true?


# In the next TWO layers there should be LSTM-s
# For them being able to be stacked, they have to give back not only the predictions at sequence end
# Somewhere there has to be a nice parameter for this... ;-)
lstm_1 = LSTM(lstm_size, return_sequences=True, use_cudnn=False)(embedding_layer)
lstm_2, state_h, state_c = LSTM(lstm_size, return_sequences=True, return_state=True, use_cudnn=False)(lstm_1)


# Important: for certain purposes (hint: search engine...) it's very useful to have the last
# hidden state and cell state also included in the results, so for the 2nd LSTM please turn
# on the return_state option. Be aware that thereby the 2nd LSTM cell will return three
# tensors (the series of outputs, last hidden state, last cell state), and you will need only
# the first of these as input for your next layer (as always, ask when in doubt)

# Finally project the output with a fully connected layer and a softmax.
output_layer = Dense(n_words + 1, activation='softmax')(lstm_2)

# What is it's width? (Help: If you have varbally elaborated well above, you already know.)


# Finally, create a model instance!
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with an appropriate optimizer and loss function
#model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 14)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 14, 100)        │      4,981,600 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 14)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 14, 128)        │        117,248 │ embedding[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 14, 128),      │        131,584 │ lstm[0][0],            │
│                           │ (None, 128), (None,    │                │ not_equal[0][0]        │
│                           │ 128)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 14, 49816)      │      6,426,264 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 11,656,696 (44.47 MB)

 Trainable params: 11,656,696 (44.47 MB)

 Non-trainable params: 0 (0.00 B)

### Error, optimizer, compilation

In [7]:
# Loss
from tensorflow.keras.losses import SparseCategoricalCrossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam, SGD, RMSprop

loss = SparseCategoricalCrossentropy # Our output is One-hot encoded. What do we use?

# Optimizer
optimizer = Adam(learning_rate=0.0001) # According to taste...

# Compilation
#############

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

### Training

We generate the trainig data.

In [8]:
data_y = np.expand_dims(data_y, -1) # It seems that Keras needs this for the "one-cold" and softmax dims to match

In [9]:
subset_size = len(data_x) // 10

data_x = data_x[:subset_size]
data_y = data_y[:subset_size]

And train!

In [10]:
# Fit a language model to the data!
# Use 10% validation - not so important in case of language models.
# Use default alidation split of Keras.
# And try to guess a realistic batch size!

batch_size = 128
epochs = 10

history = model.fit(data_x, data_y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 273ms/step - accuracy: 0.0279 - loss: 10.8145 - val_accuracy: 0.0453 - val_loss: 10.8066
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.0493 - loss: 10.7849 - val_accuracy: 0.0453 - val_loss: 10.5270
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0493 - loss: 10.2243 - val_accuracy: 0.0453 - val_loss: 9.3954
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.0493 - loss: 9.0413 - val_accuracy: 0.0453 - val_loss: 8.6098
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.0493 - loss: 8.3047 - val_accuracy: 0.0453 - val_loss: 8.1651
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.0493 - loss: 7.8783 - val_accuracy: 0.0453 - val_loss: 7.8966
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.0493 - loss: 7.6144 - val_accuracy: 0.0453 - val_loss: 7.7341
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.0493 - loss: 7.4489 - val_accuracy: 0.0

## Demo 1: Predict next word

In [11]:
# Prediction
############

def str_to_input(s):
    """Convert a string to appropriate model input.
    """
    words = [x.lower() for x in s.split()[:max_input_length]]
    ids = [br.word_to_id_map[word] for word in words]
    ids_array = np.asarray(ids)
    length = min(max_input_length, len(ids_array))
    result = np.zeros((1, max_input_length))
    result[0, :length] = ids_array[:length]
    return result, length


while True:
    s = input("\nEnter a few starting words of a sentence or <return> to stop: ")
    if s == "":
        break
    else:
        try:
            x, length = str_to_input(s)
            predictions = model.predict(x)
            probs = predictions[0][length - 1]
            most_probable = np.argmax(probs)
            print("Predicted next word:", br.id_to_word_map[most_probable])
        except KeyError:
            print("Unknown words -- please try again!")



Enter a few starting words of a sentence or <return> to stop: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted next word: the

Enter a few starting words of a sentence or <return> to stop: ho
Unknown words -- please try again!

Enter a few starting words of a sentence or <return> to stop: how
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted next word: the

Enter a few starting words of a sentence or <return> to stop: 


## Demo 2: Similarity of sentences

First we define a function that generates the hidden state of the LSTM from an input sentence:

In [17]:
import tensorflow as tf


input_layer = model.get_layer("input_layer")
lstm_2_layer = model.get_layer("lstm_1")

# Use K.function instead of keras.backend.function
cell_state_fun = tf.function(lambda x: lstm_2_layer(x)[2])

def get_embedding(x):
    """Return the final cell state associated with the input.
       Returns the last cell state as a vector.
    """
    intermediate_model = Model(inputs=model.input, outputs=[state_c])
    return intermediate_model.predict(x).flatten()

Then we use the vectors for calculating the cosine distance between sentences.

In [18]:
def cos_sim(a, b):
	return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

while True:
    s1 = input("\nEnter the first sentence or <return> to quit: ")
    if s1 == "": break
    s2 = input("\nEnter the second sentence: ")
    try:
        x1, _ = str_to_input(s1)
        x2, _ = str_to_input(s2)
        e1 = get_embedding(x1)
        e2 = get_embedding(x2)
        print("The cosine similarity between the two sentences is", cos_sim(e1, e2))
    except KeyError:
        print("Unknown words -- please try again!")


Enter the first sentence or <return> to quit: hi

Enter the second sentence: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step
The cosine similarity between the two sentences is 0.9991432

Enter the first sentence or <return> to quit: yes

Enter the second sentence: no
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
The cosine similarity between the two sentences is 0.9997914

Enter the first sentence or <return> to quit: angry

Enter the second sentence: neighbour
Unknown words -- please try again!

Enter the first sentence or <return> to quit: angry

Enter the second sentence: person


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
The cosine similarity between the two sentences is 0.99976355

Enter the first sentence or <return> to quit: yes

Enter the second sentence: person
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
The cosine similarity between the two sentences is 0.99969095

Enter the first sentence or <return> to quit: 


## Demo 3: Mini search engine

We use the library [Annoy](https://github.com/spotify/annoy) published by Spotify to create a vector space index of the Brown corpus from the LSTM's cell state. We assign a vector for each sentence, and then store it to be able to run nearest neighbor queries on it. With this we effectively created a **semantic search engine**.

(There are multiple solutions for approximate nearest neighbor search a scale which are worth looking into, one of them is [FAISS](https://code.fb.com/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) from Facebook Research.)

In [19]:
def brown_sent_to_input(ids):
  ids_array = np.asarray(ids)
  length = min(max_input_length, len(ids_array))
  result = np.zeros((1, max_input_length))
  result[0, :length] = ids_array[:length]
  return result, length

In [20]:
sentlist = list(br.sentences())

In [21]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552447 sha256=ad9b305c5e5261593ec4d7b55329a24705aaf290d032ab438a5731d6527e8e15
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [22]:
INDEX_COVERAGE_PERCENT = 0.01 #How much of the corpus you want ot index? 1.0 means whole, 0.5 means half.
NEAREST_NEIGHBOR_NUM = 5

In [24]:
from annoy import AnnoyIndex
from tqdm import tqdm

index = AnnoyIndex(128, metric="angular")

for i in tqdm(range(int(len(sentlist)*INDEX_COVERAGE_PERCENT))):
  inputs,length = brown_sent_to_input(sentlist[i])
  vector = get_embedding(inputs)
  index.add_item(i,vector)

print("Building index...")
index.build(100)
print("Index done, ready to query!")

  0%|          | 0/573 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step


  0%|          | 1/573 [00:00<07:55,  1.20it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


  0%|          | 2/573 [00:02<10:32,  1.11s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step


100%|█████████▉| 572/573 [06:09<00:00,  1.58it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step


100%|██████████| 573/573 [06:09<00:00,  1.55it/s]

Building index...
Index done, ready to query!


In [25]:
def print_brown_index(sentences, indices):
  for i in indices:
    word_ids_list = sentences[i]
    for j in word_ids_list:
      print(br.id_to_word_map[j]+" ", end='')
    print()



In [26]:
while True:
  query = input("\nEnter the query or <return> to quit: ")
  if query == "": break
  try:
    in_ids, length = str_to_input(query)
    in_vector = get_embedding(in_ids)
    nearest_sentence_indices = index.get_nns_by_vector(in_vector, NEAREST_NEIGHBOR_NUM)
    #print("nearest indices:", nearest_sentence_indices)
    print_brown_index(sentlist, nearest_sentence_indices)

  except KeyError:
    print("Unknown words -- please try again!")


Enter the query or <return> to quit: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
boonton 
oslo 
colquitt 
3 
2 

Enter the query or <return> to quit: yellow
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
2 
3 
4 
) 
oslo 

Enter the query or <return> to quit: 
